# Slippage curve demo
This notebook demonstrates alignment between empirical impact/participation data and the calibrated slippage model.

In [ ]:
import pandas as pd
from slippage import SlippageConfig, model_curve
from service_calibrate_slippage import fit_k_closed_form
from matplotlib import pyplot as plt

trades = pd.read_csv('impact_trades.csv')
k = fit_k_closed_form(trades)
cfg = SlippageConfig(k=k, default_spread_bps=float(trades['spread_bps'].median()), min_half_spread_bps=float(trades['half_spread_bps'].quantile(0.1)))
labels, bins = pd.qcut(trades['participation'], 20, labels=False, retbins=True, duplicates='drop')
curve = trades.assign(_bucket=labels).groupby('_bucket')['observed_slip_bps'].mean().to_numpy()
mids = (bins[:-1] + bins[1:]) / 2
model = model_curve(mids / trades['liquidity'].median(), cfg=cfg, spread_bps=cfg.default_spread_bps)
plt.plot(mids, curve, label='empirical')
plt.plot(mids, model, label='model')
plt.xlabel('participation')
plt.ylabel('slippage (bps)')
plt.legend()